In [79]:
import pandas as pd, numpy as np, datetime as dt
import sklearn
allProjects = pd.read_csv('allProjects.csv')
bbr = pd.read_csv('project_bandwidth.csv')
bst = pd.read_csv('project_bytes_stored.csv')
data = pd.read_pickle('userDataFree.pkl') #from GetUserLevelMasterSet-freeOnly.ipynb

In [80]:
data.drop(columns={'tier','email_hash'},axis=1, inplace=True)

In [81]:
#ignore any cases where there has been no history of downloads or uploads in a project
data2 = data[~(data.uploads==0) & ~(data.downloads==0)]

In [82]:
print(len(data),len(data2))

6799 780


In [83]:
#get a view of projects that exceed the 50GB bandwidth limit
data2.user_bw_April.sort_values(ascending=False)[0:50]

7178    2117.143406
955      673.698423
377      575.403610
3836     547.992561
1096     399.231059
1631     395.331078
1922     355.764376
4681     250.553670
7374     234.394929
4315     222.010032
2538     210.445384
506      207.898744
4526     194.156336
3048     187.249147
417      184.648257
1889     181.320951
6379     169.892121
5490     145.563046
1183     135.681660
3591     133.150382
6016     127.510956
2209     103.890411
272       98.499926
4789      98.340322
3884      87.335138
5688      80.784696
5484      78.536935
2954      74.823278
7286      73.705612
928       73.420845
6984      66.841294
4790      63.997558
1879      61.946137
1628      57.761329
5836      56.027275
1167      55.933032
973       55.710913
3245      54.750203
741       54.029279
2582      53.228576
2128      52.118098
646       52.037439
129       50.293422
1599      48.467571
6737      48.457757
5709      48.188352
6109      47.799319
144       44.470726
1558      44.384528
6916      42.536034


In [84]:
#find instances where the user has exceeded the allowable limit for either bandwidth or storage
violators = data2[(data2.user_bw_April>50) | (data2.user_storage_April>50)].user_id.unique()

In [85]:
violatorsDF = data2[(data2.user_bw_April>50) | (data2.user_storage_April>50)]

In [86]:
violators2 = violatorsDF.user_id.unique()

In [87]:
#get all projects that are registered as free-tier
allProjects = allProjects[allProjects.bandwidth_limit <=50000000000]

In [91]:
#get the project data for our
violators = allProjects[allProjects.user_id.isin(violators)]
violators2 = allProjects[allProjects.user_id.isin(violators2)]
fullgroup = allProjects[allProjects.user_id.isin(data2.user_id.unique())]
fullgroup.head()

,Unnamed: 0,created_at,project_id,user_id,usage_limit,bandwidth_limit
14,14,2021-04-24,01466c3e63784efb81f2e0e4f6f0ec36,3a731a864fab4301865f7d3b69dcb2e9,50000000000,50000000000
15,15,2021-04-12,016c3243acc14467a441996c408a7cc1,abb2d2a3b56544f7a334d97d7736decc,50000000000,50000000000
44,44,2021-04-23,049a555d556a403b9113ba6224244355,1da30f30b6114118adbf14e13b1e8e6e,50000000000,50000000000
63,63,2021-04-23,067b9a1733c34739be924eb447bd1e81,15568278de60427dbc5dc0af0302d640,50000000000,50000000000
65,65,2021-04-29,06cc486ebd9549cdb3f3ec4ec6e95ad7,5ab2de0479314b1f97bd58c4780272f7,50000000000,50000000000


In [92]:
#get the unique project_ids of the violators
violatorsProj = violators.project_id.unique()
violatorsProj2 = violators2.project_id.unique()
fullgroup = fullgroup.project_id.unique()

In [93]:
#get their corresponding bandwidth usage history
violatorsProjbbr = bbr[bbr.project_id.isin(violatorsProj)]
violatorsProjbbr2 = bbr[bbr.project_id.isin(violatorsProj2)]
fullgroupbbr = bbr[bbr.project_id.isin(fullgroup)]

In [94]:
#tag each transaction by month and year
violatorsProjbbr.interval_start = pd.to_datetime(violatorsProjbbr.interval_start)
violatorsProjbbr['period'] = violatorsProjbbr.interval_start.dt.to_period('M')

violatorsProjbbr2.interval_start = pd.to_datetime(violatorsProjbbr2.interval_start)
violatorsProjbbr2['period'] = violatorsProjbbr2.interval_start.dt.to_period('M')

fullgroupbbr.interval_start = pd.to_datetime(fullgroupbbr.interval_start)
fullgroupbbr['period'] = fullgroupbbr.interval_start.dt.to_period('M')

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [95]:
#get total amount of bandwidth used by project by month
violatorhistory = violatorsProjbbr.groupby(['project_id','period'])['settled'].sum()
violatorhistory2 = violatorsProjbbr2.groupby(['project_id','period'])['settled'].sum()
fullgrouphistory = fullgroupbbr.groupby(['project_id','period'])['settled'].sum()

In [96]:
#view
pd.set_option('max_rows', 800)
violatorhistory = violatorhistory.to_frame().unstack().fillna(0)
violatorhistory.columns = violatorhistory.columns.droplevel(0)

violatorhistory2 = violatorhistory2.to_frame().unstack().fillna(0)
violatorhistory2.columns = violatorhistory2.columns.droplevel(0)
violatorhistory2

fullgrouphistory = fullgrouphistory.to_frame().unstack().fillna(0)
fullgrouphistory.columns = fullgrouphistory.columns.droplevel(0)

In [97]:
violatorhistoryx=violatorhistory.reset_index()
violatorhistoryx2=violatorhistory2.reset_index()
fullgrouphistory=fullgrouphistory.reset_index()
violatorhistoryx2


period,project_id


In [56]:
violatorhistoryx

period,project_id,2021-04,2021-05,2021-06
0,0f330c39974e4d1d978866d0db33574b,1.197196e+11,0.000000e+00,0.000000e+00
1,0f9abf82930c4a07a3e9a42b75c16ee0,8.733514e+10,6.656000e+03,0.000000e+00
2,11239b11a70c42c8bbe2ebae4c600aa6,7.370561e+10,2.854148e+10,6.594937e+10
3,1b3e9cd5494f4991819e4dbab079c1b2,1.912811e+11,1.851325e+11,0.000000e+00
4,1b99d5f5739d4e27a11b61775498385b,1.878765e+11,0.000000e+00,0.000000e+00
5,1c9eef5ec6bf46f0b499a4f635bef5a8,2.254125e+11,6.684704e+09,0.000000e+00
6,1fddc44531624635aa3e20050a4ea322,1.698921e+11,0.000000e+00,0.000000e+00
7,209c83e68a544a5b9780d262b62cbf3e,5.593303e+10,8.177932e+09,5.128448e+06
8,2206b7bd93c14b91954a31b10b4581e7,1.038904e+11,4.511170e+11,1.223091e+11
9,227cdf1c8f9f4d62b9b552e45a238de7,6.194614e+10,1.202592e+11,1.720787e+10


In [16]:
violatorhistoryx[violatorhistoryx.project_id=='d829c2f4ef344d73897abd765dc07b27']

period,project_id,2021-04,2021-05,2021-06
53,d829c2f4ef344d73897abd765dc07b27,9.849993e+10,0.0,0.0


In [41]:
violatorhistoryx

period,project_id


In [37]:
pd.set_option('max_rows', 2000)
checker = bbr[bbr.project_id=='7469f059951c4e27964002158e588a4d'].reset_index()
print('uploads: ',checker[checker.action==1].settled.sum()/1000000000)
print('doloads: ',checker[checker.action==2].settled.sum()/1000000000)

uploads:  9009.26500608
doloads:  37.194526976


In [64]:
def addition(id, type):
    checker = bbr[bbr.project_id==id].reset_index()
    if type=='upload':
        return checker[checker.action==1].settled.sum()/1000000000
    elif type=='download':
        return checker[checker.action==2].settled.sum()/1000000000
    

In [65]:
checkerNew = bbr[bbr.project_id.isin(violatorhistoryx.project_id)]
violatorhistoryx['totalUp']= violatorhistoryx.apply(lambda x: addition(x['project_id'],'upload'),axis=1)
violatorhistoryx['totalDown']= violatorhistoryx.apply(lambda x: addition(x['project_id'],'download'),axis=1)
violatorhistoryx['tally']= violatorhistoryx['totalUp']-violatorhistoryx['totalDown']

#(lambda x: checkerNew[checkerNew.project_id==x.project_id].settled.sum()/1000000000 if x.action==1 else 0, axis=1)
violatorhistoryx

period,project_id,2021-04,2021-05,2021-06,totalUp,totalDown,tally
0,0f330c39974e4d1d978866d0db33574b,1.197196e+11,0.000000e+00,0.000000e+00,74.053922,45.665682,28.388240
1,0f9abf82930c4a07a3e9a42b75c16ee0,8.733514e+10,6.656000e+03,0.000000e+00,86.424249,0.910896,85.513352
2,11239b11a70c42c8bbe2ebae4c600aa6,7.370561e+10,2.854148e+10,6.594937e+10,163.061535,5.134928,157.926607
3,1b3e9cd5494f4991819e4dbab079c1b2,1.912811e+11,1.851325e+11,0.000000e+00,295.927716,80.485877,215.441839
4,1b99d5f5739d4e27a11b61775498385b,1.878765e+11,0.000000e+00,0.000000e+00,187.876466,0.000000,187.876466
5,1c9eef5ec6bf46f0b499a4f635bef5a8,2.254125e+11,6.684704e+09,0.000000e+00,206.133244,25.963913,180.169331
6,1fddc44531624635aa3e20050a4ea322,1.698921e+11,0.000000e+00,0.000000e+00,169.892092,0.000029,169.892063
7,209c83e68a544a5b9780d262b62cbf3e,5.593303e+10,8.177932e+09,5.128448e+06,64.014485,0.101607,63.912878
8,2206b7bd93c14b91954a31b10b4581e7,1.038904e+11,4.511170e+11,1.223091e+11,670.482294,6.834154,663.648140
9,227cdf1c8f9f4d62b9b552e45a238de7,6.194614e+10,1.202592e+11,1.720787e+10,198.843287,0.569943,198.273344


In [99]:
# check to see if there are any other projects where download bw exceeds upload bw amoung accounts that 
# didn't violate the 50GB limits.
checkerFull = bbr[bbr.project_id.isin(fullgrouphistory.project_id)]
fullgrouphistory['totalUp']= fullgrouphistory.apply(lambda x: addition(x['project_id'],'upload'),axis=1)
fullgrouphistory['totalDown']= fullgrouphistory.apply(lambda x: addition(x['project_id'],'download'),axis=1)
fullgrouphistory['tally']= fullgrouphistory['totalUp']-fullgrouphistory['totalDown']

period,project_id,2020-12,2021-04,2021-05,2021-06,totalUp,totalDown,tally
9,0217ceb8396f460087c6c2fd05189234,0.0,2.464768e+07,2.033254e+07,0.000000e+00,0.016043,0.028937,-0.012894
24,08360cf38b564471a818329ccb2d7e7e,0.0,1.008640e+07,0.000000e+00,0.000000e+00,0.004830,0.005256,-0.000426
28,09622a23e10643ef887fba9925f85536,0.0,8.430080e+05,0.000000e+00,0.000000e+00,0.000375,0.000468,-0.000094
31,0a16eeaecbd14300a3758a60958fcc45,0.0,4.649267e+07,0.000000e+00,0.000000e+00,0.019240,0.027253,-0.008013
32,0a44858cb3fe4b7c806c22a9485b8e48,0.0,2.312292e+08,0.000000e+00,0.000000e+00,0.075398,0.155831,-0.080433
35,0b603ac0daf94064b027ac473c4893a8,0.0,1.015526e+07,0.000000e+00,0.000000e+00,0.004164,0.005992,-0.001828
38,0cb694383039456abc20469751677d3c,0.0,9.205760e+05,0.000000e+00,0.000000e+00,0.000443,0.000478,-0.000035
39,0ce0984a6d11404f9fc0e9e500a498a8,0.0,8.768000e+05,0.000000e+00,0.000000e+00,0.000223,0.000653,-0.000430
41,0d546c3aae9544dea44ace9c822ef3d1,0.0,4.097664e+07,0.000000e+00,0.000000e+00,0.015897,0.025080,-0.009182
54,11dc5346c1844df8ac812de106a7f788,0.0,1.223910e+07,0.000000e+00,0.000000e+00,0.005944,0.006296,-0.000352


In [103]:
possibleCheaters = fullgrouphistory[fullgrouphistory['tally']<0]
print(len(possibleCheaters))
possibleCheaters.rename(columns={'totalUp':'totalUploadBandwidth','totalDown':'totalDownloadBandwidth','tally':'difference'},inplace=True)
possibleCheaters=possibleCheaters[['project_id','totalUploadBandwidth','totalDownloadBandwidth','difference']] 
possibleCheaters[possibleCheaters['difference']<-8]

108


C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\frame.py:4298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


period,project_id,totalUploadBandwidth,totalDownloadBandwidth,difference
83,1aa59cdd5dff46f19b26bee4e1c0ee25,8.688674,18.130097,-9.441423
112,22c86abe69a845178764dec325a59dfe,15.940840,76.280215,-60.339375
360,6b2d0a6d00c44b5abd88758afc2e4798,11.000226,44.606494,-33.606268
823,f7afe39641eb4f2283546b6bd1db311b,9.985495,27.828920,-17.843425


In [106]:
#confirming the projects belong to different users
projectsAll[projectsAll.project_id.isin(possibleCheaters[possibleCheaters['difference']<-8].project_id)].user_id

1137     70f712a69889425b9b1954766466f3bf
3406     c77abf0b87984917a12da605926888aa
9870     91efd1c314b9468392f7e6a5844dfefc
10977    04e126ce1b294ecd9c2f566de0412c67
Name: user_id, dtype: object

In [105]:
projectsAll = pd.read_csv('allProjects.csv')
projectsAll[projectsAll.project_id=='d829c2f4ef344d73897abd765dc07b27']

,Unnamed: 0,created_at,project_id,user_id,usage_limit,bandwidth_limit
15452,5306,2021-04-28,d829c2f4ef344d73897abd765dc07b27,0a432bced77c497cbf4f014621bd8e43,50000000000,50000000000


In [19]:
projectsAll[projectsAll.project_id=='553643593cac45979c57af9508742590']


,Unnamed: 0,created_at,project_id,user_id,usage_limit,bandwidth_limit
897,897,2021-04-27,553643593cac45979c57af9508742590,691817630d4746ca854a09607e46906d,50000000000,50000000000


In [20]:
violatorhistoryx[violatorhistoryx.project_id=='553643593cac45979c57af9508742590']

period,project_id,2021-04,2021-05,2021-06
21,553643593cac45979c57af9508742590,1.872491e+11,1.026204e+10,1.217250e+10


In [26]:
violatorsProjbst = bst[bst.project_id.isin(violatorsProj)]
violatorsProjbst.interval_start = pd.to_datetime(violatorsProjbst.interval_start)
violatorsProjbst['period'] = violatorsProjbst.interval_start.dt.to_period('M')
violatorhistoryStorage = violatorsProjbst.groupby(['project_id','period'])['remote'].max()
violatorhistoryStorage = violatorhistoryStorage.to_frame().unstack().fillna(0)
violatorhistoryStorage.columns = violatorhistoryStorage.columns.droplevel(0)
violatorhistoryStorage = violatorhistoryStorage.reset_index()
violatorhistoryStorage[violatorhistoryStorage.project_id=='d829c2f4ef344d73897abd765dc07b27']

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


period,project_id,2021-04,2021-05,2021-06
52,d829c2f4ef344d73897abd765dc07b27,2.795678e+10,2.795678e+10,0.0


In [24]:
violatorhistoryStorage

period,2021-04,2021-05,2021-06
project_id,,,
0f330c39974e4d1d978866d0db33574b,2.094914e+10,2.094914e+10,2.094914e+10
0f9abf82930c4a07a3e9a42b75c16ee0,2.125698e+10,2.125698e+10,2.125698e+10
11239b11a70c42c8bbe2ebae4c600aa6,1.912258e+10,2.269806e+10,3.504034e+10
1b3e9cd5494f4991819e4dbab079c1b2,2.008311e+10,2.853388e+10,8.054001e+09
1b99d5f5739d4e27a11b61775498385b,4.945514e+10,4.945514e+10,4.945514e+10
1c9eef5ec6bf46f0b499a4f635bef5a8,1.890006e+10,6.092483e+09,6.092483e+09
1fddc44531624635aa3e20050a4ea322,4.632489e+10,4.632489e+10,4.632489e+10
209c83e68a544a5b9780d262b62cbf3e,1.494231e+10,1.714655e+10,1.714655e+10
2206b7bd93c14b91954a31b10b4581e7,2.903380e+10,3.168712e+10,2.412540e+10
